In [2]:
import numpy as np, os
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

# Data Load

In [3]:
import random    
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

# MAE (평균절대오차)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

seed_num = 42
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[early_stop], shuffle=False)

# COLS
a = pd.read_csv('total_data.csv')
COLS = list(a['ITEMID'].sort_values().unique())

results=[]

gpu_strategy = tf.distribute.get_strategy()
with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE MAE WITH FEATURE K SHUFFLED
        preds = lstm.predict(X_test, verbose=0)
        mae = np.mean(np.abs(preds-y_test))

        results.append({'feature':COLS[k],'mae':mae})
        X_test[:,:,k] = save_col
        
        # DISPLAY LSTM FEATURE IMPORTANCE
        df = pd.DataFrame(results)
        df = df.sort_values('mae')

Epoch 1/500
37/37 [==============================] - 13s 88ms/step - loss: 0.6801 - acc: 0.5915 - val_loss: 0.6654 - val_acc: 0.6177
Epoch 2/500
37/37 [==============================] - 2s 54ms/step - loss: 0.6742 - acc: 0.6036 - val_loss: 0.6482 - val_acc: 0.6177
Epoch 3/500
37/37 [==============================] - 2s 54ms/step - loss: 0.6379 - acc: 0.6298 - val_loss: 0.5525 - val_acc: 0.7354
Epoch 4/500
37/37 [==============================] - 2s 54ms/step - loss: 0.5246 - acc: 0.7497 - val_loss: 0.5088 - val_acc: 0.7613
Epoch 5/500
37/37 [==============================] - 2s 53ms/step - loss: 0.4379 - acc: 0.8068 - val_loss: 0.5021 - val_acc: 0.7697
Epoch 6/500
37/37 [==============================] - 2s 53ms/step - loss: 0.3770 - acc: 0.8549 - val_loss: 0.5496 - val_acc: 0.7691
Epoch 7/500
37/37 [==============================] - 2s 52ms/step - loss: 0.3455 - acc: 0.8682 - val_loss: 0.5499 - val_acc: 0.7652
Epoch 8/500
37/37 [==============================] - 2s 53ms/step - loss: 0

100%|██████████| 4069/4069 [40:05<00:00,  1.69it/s]


In [7]:
df

,feature,mae
121,51006,0.471799
206,51277,0.471801
56,50912,0.472565
34,50882,0.472682
26,50868,0.473135
...,...,...
19,50861,0.474062
174,51221,0.474088
187,51249,0.474104
182,51237,0.474128


# Binary Crossentropy
- 학습 시 loss에 'binary_crossentropy'대신 tf.keras.losses.BinaryCrossentropy()
- LSTM2

## seed_num = 42

In [ ]:
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)

lstm2 = Sequential()
lstm2.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm2.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm2.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm2.add(Dropout(0.2))
lstm2.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm2.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm2.add(Dropout(0.2))
lstm2.add(Dense(units=1, activation='sigmoid'))

ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm2.compile(optimizer= "adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['acc'])
lstm2.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[early_stop], shuffle=False)

# COLS
a = pd.read_csv('total_data.csv')
COLS = list(a['ITEMID'].sort_values().unique())

results = []
preds = lstm2.predict(X_test, verbose=0)

# COMPUTE BASELINE (NO SHUFFLE)
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

gpu_strategy = tf.distribute.get_strategy()
with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        preds = lstm2.predict(X_test, verbose=0)
        loss_bce = bce(y_test, preds).numpy()

        results.append({'feature':COLS[k],'bce':loss_bce})
        X_test[:,:,k] = save_col

        # DISPLAY LSTM FEATURE IMPORTANCE
        df = pd.DataFrame(results)

## seed_num = 43

In [12]:
import random    
# ---------------------
seed_num = 44
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

# ---------------------
seed_num = 44
# ---------------------
tf.random.set_seed(seed_num)

lstm2 = Sequential()
lstm2.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm2.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm2.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm2.add(Dropout(0.2))
lstm2.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm2.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm2.add(Dropout(0.2))
lstm2.add(Dense(units=1, activation='sigmoid'))

ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm2.compile(optimizer= "adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['acc'])
lstm2.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[early_stop], shuffle=False)

# COLS
a = pd.read_csv('total_data.csv')
COLS = list(a['ITEMID'].sort_values().unique())

results = []
preds = lstm2.predict(X_test, verbose=0)

# COMPUTE BASELINE (NO SHUFFLE)
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

gpu_strategy = tf.distribute.get_strategy()
with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        preds = lstm2.predict(X_test, verbose=0)
        loss_bce = bce(y_test, preds).numpy()

        results.append({'feature':COLS[k],'bce':loss_bce})
        X_test[:,:,k] = save_col

        # DISPLAY LSTM FEATURE IMPORTANCE
        df = pd.DataFrame(results)

Epoch 1/500
37/37 [==============================] - 8s 81ms/step - loss: 0.6778 - acc: 0.5840 - val_loss: 0.6668 - val_acc: 0.6138
Epoch 2/500
37/37 [==============================] - 2s 53ms/step - loss: 0.6729 - acc: 0.6126 - val_loss: 0.6514 - val_acc: 0.6138
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6251 - acc: 0.6507 - val_loss: 0.5289 - val_acc: 0.7393
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.4807 - acc: 0.7853 - val_loss: 0.5288 - val_acc: 0.7316
Epoch 5/500
37/37 [==============================] - 2s 53ms/step - loss: 0.4315 - acc: 0.8126 - val_loss: 0.4821 - val_acc: 0.7762
Epoch 6/500
37/37 [==============================] - 2s 53ms/step - loss: 0.3618 - acc: 0.8584 - val_loss: 0.5011 - val_acc: 0.7736
Epoch 7/500
37/37 [==============================] - 2s 52ms/step - loss: 0.3250 - acc: 0.8795 - val_loss: 0.5261 - val_acc: 0.7723
Epoch 8/500
37/37 [==============================] - 2s 52ms/step - loss: 0.

100%|██████████| 4069/4069 [44:44<00:00,  1.52it/s]


In [21]:
df1 = df.copy()
df1['baseline_bce'] = float(df['baseline_bce'].dropna().unique())
df1['diff'] = df1['bce']-df1['baseline_bce']
df1 = df1.sort_values(by='diff')
df1

,feature,baseline_bce,bce,diff
3834,63323026201,0.961885,0.952192,-0.009693
122,51006,0.961885,0.952638,-0.009247
207,51277,0.961885,0.956470,-0.005415
35,50882,0.961885,0.957884,-0.004001
3458,55390000401,0.961885,0.958644,-0.003241
...,...,...,...,...
2764,10432017002,0.961885,0.962698,0.000813
2312,517260225,0.961885,0.962861,0.000976
2293,487980125,0.961885,0.962921,0.001036
1831,182853489,0.961885,0.963312,0.001427
